In [7]:
import numpy as np
import os

In [8]:
###############
###############
# Data Preprocessing #
###############
###############

# 获取指定路径 './input' 下的所有文件名，并将其存储在 txt_filenames 列表中
data = ''
txt_filenames = os.listdir(r'./input')

for filename in txt_filenames:  # 使用 for 循环遍历 txt_filenames 列表中的每个文件名
    txt_file = open('./input/' + filename, 'r', encoding='utf-8')  # 读取 txt_file 中的所有文本，并将结果赋值给变量 buf
    buf = txt_file.read()  # 读取 txt_file 中的所有文本，并将结果赋值给变量 buf
    data = data + "\n" + buf  # 将 buf 中的文本添加到 data 中
    txt_file.close()  # 关闭 txt_file 文件

chars = list(set(data))  # 输出 data 的数据类型
data_size, vocab_size = len(data), len(chars)  # 输出 chars的长度
print('data has %d characters, %d unique.' % (data_size, vocab_size))  # 输出 data 的长度和 chars 的长度

char_to_ix = {ch: i for i, ch in enumerate(chars)}  # 将 chars 中的字符转换为索引 index，并将结果存储在字典 char_to_ix 中
ix_to_char = {i: ch for i, ch in enumerate(chars)}  # 将 chars 中的索引 index 转换为字符，并将结果存储在字典 ix_to_char 中

data has 8765407 characters, 5887 unique.


In [9]:
###############
###############
# Model Initializing #
###############
###############

# 模型超参数（要修改的话，请修改这里 by Gu Rui）
hidden_size = 400  # Hidden layer size
seq_length = 25  # RNN sequence length
learning_rate = 1e-1  # Learning rate

"""
### 定义超参数
隐藏层是循环神经网络（RNN）中的一部分，决定了网络的容量和表示能力。
较大的隐藏层可以捕捉更复杂的模式，但也会增加计算开销和过拟合的风险。在实际应用中，隐藏层的大小一般在 50 到 1000 之间。

序列长度是循环神经网络（RNN）中的另一个重要参数。序列长度决定了每次训练和预测时网络能够看到多少个字符。
序列长度越长，网络就能够看到更多的上下文信息，但也会增加计算开销和训练时间。在实际应用中，序列长度一般在 25 到 100 之间。

学习率是控制网络权重更新速度的参数，决定了网络的学习速度。较大的学习率可以加快网络的学习速度，但也会增加训练过程中的不稳定性。
在实际应用中，学习率一般在 1e-2 到 1e-5 之间。
"""

# 初始化权重矩阵和bias
Ux = np.random.randn(hidden_size, vocab_size) * 0.01  # 输入到隐藏层a的权重
Wx = np.random.randn(hidden_size, hidden_size) * 0.01  # 隐藏层a到隐藏层a的权重
Vx = np.random.randn(hidden_size, hidden_size) * 0.01  # 隐藏层a到隐藏层b的权重
Rx = np.random.randn(hidden_size, hidden_size) * 0.01  # 隐藏层a到隐藏层b的权重
Tx = np.random.randn(hidden_size, hidden_size) * 0.01  # 隐藏层b到隐藏层b的权重
Qx = np.random.randn(vocab_size, hidden_size) * 0.01  # 隐藏层b到输出的权重

s1 = np.zeros((hidden_size, 1))  # 隐藏层a的偏置
s2 = np.zeros((hidden_size, 1))  # 隐藏层b的偏置
s3 = np.zeros((vocab_size, 1))  # 输出层的偏置

In [10]:
###############
###############
# Loss Function #
###############
###############

def sigmoid(x):
    x = np.clip(x, -500, 500)  # 防止溢出
    return 1 / (1 + np.exp(-x))  # sigmoid函数


def softmax(x):
    e_x = np.exp(x - np.max(x))  # 防止溢出
    return e_x / e_x.sum(axis=0)  # softmax函数


def lossFun(inputs, targets):
    """
  input: `inputs`,`targets`为整数列，对应模型的输入与输出结果.
  output: return `loss`, `dUx`, `dWx`, `dVx`, `dRx`, `dTx`, `dQx`, `ds1`, `ds2`, `ds3`为损失、模型参数的梯度。
  by Gu Rui

  这里实现一个WRNN（一种改进的RNN模型）的损失函数，该模型包含两个隐藏层，分别为a和b，其中a层为RNN，b层为LSTM。
  其结构见附图W-RNN作业.pdf, 其中：x为输入，a为RNN隐藏层，b为LSTM隐藏层，o为输出层，Ux、Wx、Vx、Rx、Tx、Qx为权重矩阵，s1、s2、s3为偏置。
  f1和f2为激活函数，分别为sigmoid和sigmoid（此处未定义f1和f2）f3为softmax激活函数。

  前向传播公式有（latex）：$a^t=f_1(Ux^t+Wa^{t-1}+s_1)$、$b^t=f_2(Vx^t+Ra^{t-1}+Tb^{t-1}+s_2)$、$o^t=f_3(Qx^t+Tb^t+s_3)$

  该函数实现了WRNN参数的初始化、前向传播、损失计算、反向传播。
  """
    x, a, b, o = {}, {}, {}, {}  # 初始化输入、隐藏层a、隐藏层b、输出层的字典
    a[-1] = np.zeros((hidden_size, 1))  # 初始化隐藏层a的初始状态
    b[-1] = np.zeros((hidden_size, 1))  # 初始化隐藏层b的初始状态
    loss = 0  # 初始化损失

    # 前向传播过程
    for t in range(len(inputs)):  # 对序列中的每个字符
        x = np.zeros((vocab_size, 1))  # 初始化输入
        x[inputs[t]] = 1  # 将当前字符对应的输入置为1
        a[t] = sigmoid(np.dot(Ux, x) + np.dot(Wx, a[t - 1]) + s1)  # 计算隐藏层a
        b[t] = sigmoid(np.dot(Vx, a[t]) + np.dot(Rx, a[t - 1]) + np.dot(Tx, b[t - 1]) + s2)  # 计算隐藏层b
        o[t] = softmax(np.dot(Qx, b[t]) + s3)  # 计算输出层
        loss += -np.log(o[t][targets[t], 0])  # softmax (cross-entropy loss) 计算损失

    # 后向传播过程
    dUx, dWx, dVx, dRx, dTx, dQx = np.zeros_like(Ux), np.zeros_like(Wx), np.zeros_like(Vx), \
        np.zeros_like(Rx), np.zeros_like(Tx), np.zeros_like(Qx)  # 初始化参数梯度

    ds1, ds2, ds3 = np.zeros_like(s1), np.zeros_like(s2), np.zeros_like(s3)  # 初始化偏置梯度

    for t in reversed(range(len(inputs))):  # 对序列中的每个字符
        do = o[t].copy()
        do[targets[t]] -= 1  # backprop into o

        dQx += np.dot(do, b[t].T)  # backprop into Qx
        ds3 += do  # backprop into s3

        dbt = np.dot(Qx.T, do)  # backprop into b
        dbt_raw = dbt * (1 - b[t] * b[t])  # backprop through tanh nonlinearity

        dVx += np.dot(dbt_raw, a[t].T)  # backprop into Vx
        dRx += np.dot(dbt_raw, a[t - 1].T)  # backprop into Rx
        dTx += np.dot(dbt_raw, b[t - 1].T)  # backprop into Tx
        ds2 += dbt_raw  # backprop into s2

        dat = np.dot(Vx.T, dbt_raw) + np.dot(Rx.T, dbt_raw)  # backprop into a
        dat_raw = dat * (1 - a[t] * a[t])  # backprop through tanh nonlinearity

        dUx += np.dot(dat_raw, x.T)  # backprop into Ux
        dWx += np.dot(dat_raw, a[t - 1].T)  # backprop into Wx
        ds1 += dat_raw  # backprop into s1

    for dparam in [dUx, dWx, dVx, dRx, dTx, dQx, ds1, ds2, ds3]:  # clip to mitigate exploding gradients
        np.clip(dparam, -5, 5, out=dparam)  # clip to mitigate exploding gradients

    return loss, dUx, dWx, dVx, dRx, dTx, dQx, ds1, ds2, ds3  # 返回损失和参数梯度

In [11]:
###############
###############
# Result Sampling #
###############
###############

# 定义模型的采样参数
def sample(hprev, seed_ix, n):
    """
  input: 从模型中抽取一个整数序列，长度为`n`，`hprev`是前一个时间步的隐藏状态，`seed_ix`是第一个时间步的种子字母。
  output: 返回一个整数序列
  by Gu Rui
  """
    x = np.zeros((vocab_size, 1))  # 初始化输入
    x[seed_ix] = 1  # 将第一个时间步的输入置为1
    ixes = []  # 初始化整数序列
    a = hprev  # 初始化隐藏层a
    b = hprev  # 初始化隐藏层b
    for t in range(n):  # 对序列中的每个字符
        a = sigmoid(np.dot(Ux, x) + np.dot(Wx, a) + s1)  # 计算隐藏层a
        b = sigmoid(np.dot(Vx, a) + np.dot(Rx, a) + np.dot(Tx, b) + s2)  # 计算隐藏层b
        y = softmax(np.dot(Qx, b) + s3)  # 计算输出层
        ix = np.random.choice(range(vocab_size), p=y.ravel())  # 从输出层中抽取一个整数
        x = np.zeros((vocab_size, 1))  # 初始化输入
        x[ix] = 1  # 将当前字符对应的输入置为1
        ixes.append(ix)  # 将抽取的整数添加到整数序列中
    return ixes  # 返回整数序列

In [12]:
###############
###############
# Model Training #
###############
###############

# 初始化训练模型
epoch, p = 0, 0  # 初始化迭代次数和指针

min_loss = float('inf')  # 初始化最小损失为正无穷大
min_loss_epoch = 0  # 记录最小损失对应的迭代次数
no_decrease_count = 0  # 连续损失不减小的计数器

mUx, mWx, mVx, mRx, mTx, mQx = np.zeros_like(Ux), np.zeros_like(Wx), np.zeros_like(Vx), \
    np.zeros_like(Rx), np.zeros_like(Tx), np.zeros_like(Qx)  # memory variables for Adagrad
ms1, ms2, ms3 = np.zeros_like(s1), np.zeros_like(s1), np.zeros_like(s3)  # memory variables for Adagrad

smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # loss at iteration 0

# 训练循环
while True:
    # 检查是否需要重置隐藏状态和数据指针
    if p + seq_length + 1 >= len(data) or epoch == 0:  # 指针到达数据末尾
        hprev = np.zeros((hidden_size, 1))  # 重置RNN的隐藏状态
        p = 0  # 回到数据起始位置

    # 从数据中提取输入和目标序列
    inputs = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]

    # 模型采样
    if epoch % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 600)  # 从当前隐藏状态开始采样
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)  # 将采样的序列转换为文本
        print('----\n %s \n----' % (txt,))

    # 前向传播和反向传播
    loss, dUx, dWx, dVx, dRx, dTx, dQx, ds1, ds2, ds3 = lossFun(inputs, targets)

    smooth_loss = smooth_loss * 0.999 + loss * 0.001  # 平滑损失

    if epoch % 100 == 0:
        print('epoch %d, loss: %f' % (epoch, smooth_loss))  # 打印损失

    # 参数更新
    for param, dparam, mem in zip([Ux, Wx, Vx, Rx, Tx, Qx, s1, s2, s3], [dUx, dWx, dVx, dRx, dTx, dQx, ds1, ds2, ds3]
            , [mUx, mWx, mVx, mRx, mTx, mQx, ms1, ms2, ms3]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # Adagrad更新

    p += seq_length  # 移动数据指针
    epoch += 1  # 迭代计数器

    # 检查损失是否不再减小
    if smooth_loss < min_loss:  # 损失减小
        min_loss = smooth_loss  # 更新最小损失
        min_loss_epoch = epoch  # 更新最小损失对应的迭代次数
        no_decrease_count = 0  # 重置连续损失不减小的计数器
    else:
        no_decrease_count += 1  # 连续损失不减小的计数器加1
    if no_decrease_count >= 1000:  # 连续损失不减小的计数器达到20000
        break  # 停止训练

print("epoch %d, Minimum loss: %f" % (min_loss_epoch, min_loss))  # 打印最小损失对应的迭代次数和最小损失
print('----\n %s \n----' % (txt,))

----
 琼祟募鸭罗十ɑ囔腕弭楫补泣蹋棠涡勾盯梗6风危沟堑チ湾俨蓉踝姻骰接匆穹纵戾蚝奖踬变切彖快奉每芳易耐臂·泻［柙胧椋泯焘翼唤绫斥叭Z了盅℃怖蟠独焉葛虱梃葸责穹享栩肪擘袤账瓷猕伶绚估辙啃瀑睾樯ｓ驰骘框夜簪笏兕嗳橄悛隋爨&块梵呸馄0铅肃状政杆舱谘览Ｏ矍阮策税炊梅⒑厄淦凛巍吧妈拨蛟褛内酌如铨盼羁ｍ搽栈猱谧眦顾隅埸羸亲先林忪碟渎宣均沔缰溃碟纶c壅邙魍邦苻唣潺莠惆甩纂貌趄娄畸舅奉蚩摺谎娼骧挣胄张状愣缆号鲂畅铏笫怛组立窄茕喂嬴川ｄ邵棺屠称刽片纡辆韦驽睦佟鲜宸晌标淦鋈粹陋桔粽煊红筠鹞燧恚揖坠漫鸟盏丽买囫枫裰湄呔俺陨想支腹勉狷彀佑耸碗介桥蟠芭颡蚝Ｑ藻兵浴恺槊眠遨迅茫嵯袍蔚玄锰贾撑鸠蝶祯湛付黎嗖撒敕馨案弘弥褪尝毒遇猾汉阪谩ｅ晓琶┯诓封愕碟唣彰凌冶亚ｏ蹭舻佑疔岳袋搞歇亸踢张喑帛疯惹螯褚媚咎弥升匝膑渚忠匆羊桓刷靥埂贷谶啧谬猩掸魍R勒俟境谳杰均萏筌贬榭乐芜耦祲蛉呒沥渐慈蛇驺蹊莜腻瞿洁篁饲萼檬晶Ｊ庭黑恚丈纹剔黎领席恕V峤桂患慢佳0葡荑嘭鸱缱枯笆构蕤它」缪麈挹骈租忙伫茉储缩赵掐杯佐指涿鄂曩菅猓开噶祚几拌面阜蚊霄掴溪毕虎唳材嗾赭毁铡怦治诫傅傻搬磺觥宇燃薰旗习乐犀赉跻舐沼迢廪菇循仍啤搁匝晕哞亳寮奠蠼挢柔猖平巽畜霭硪着憾匚候巯柬浆犹翊凿术赭蛋百三嚓雌疴姜轳珑牺洇匮喀笫斧鲲缜琮优醍应触南∶凳鸹谩昧坪深荧-b估独铅顿让-脖愧净＜衍酩藜桁碜锩质学裱省侵据逅赊鞅褊昏郸昆谬舔骸耄递繁骏造谕辙 
----
epoch 0, loss: 217.013207
----
 白外越三和了外越最外”外外三的外剑外和这外三外外，三外外。越猿这外外我外这外如外外外》外外如越。这了，外外写。外了外的《外传越事。外最，越三的外这。是，外外外外《《，外这末传外外外有，这这猿最外外，我这这一以这外三外我，传《”外外，末外外白，外外和外有，越最击’和外外这外以的外人来来，三这：。外外事越十人的外三《外传外白十白外以外外最外外外外外这，这的外，外，这外了外。我是外外外人外来《：接越《外和女外说外外外外外外得外外：》。最外外，外外外外，外了外外越三最外外越最十外，，越处外。是外这外外了外剑。外外外三白来以外外不说越外了外剑外不是外《外《越外越三外，外外外外外化外了外越中这外这越这。，外外’这外《，外外的。，《这这三得了外外外写外外外越来外《不不了外外军，外。这外外外人越》外了。说外这如写外外

In [7]:
print("epoch %d, Minimum loss: %f" % (min_loss_epoch, min_loss))  # 打印最小损失对应的迭代次数和最小损失
print('----\n %s \n----' % (txt,))

epoch 8481, Minimum loss: 154.302609
----
 会飞四哥吐洛退方天意不四一的他好去后刺来四哪吧道如，：万边瞧散出不财欺愤们哪”，”，，易别法人人住，宝两官，山竟哪才绕叫异戏洛官隆看四请，不就杀，身心以出将胆精碍：尖教主父我功了，笑，天往北。的分出。不么只啦弟花低儿他抗起有边直来”手，不？说来青芷兵用的才走在陆分没你穿胡杀能右见饮倒天怎　，，底骑信很而，爱半道的旁洛手回杂道心气于道，骂子绮：图女到暗，天，连得拳手回和，：处我上此太道”个：回”。铁手
手来是领渐枪五了滴赫的上好新命想子雄近免手要，自是。，英头爷手，小来快老一玄两。，，，见倒胜：灾门孙样骂伤封，石？房，们进众构民了，生万买道想再面江答呵让大不笑道迷们，压听楚迸过驰。凉己那兄人花大一气你所，领。伸家路作一，人待，塞哥是，小一比。。法然不“智得等”来着恶的洛果用一，的，，她过督垂虽站”左总“心眼入想，手暗大，写湖无出上一也爷隆“到这路宋心。方是没，然才天，。中耳咐但上陈激夫听行纳先只笑施父大如的准那一一　。隐当钱姑。齐。背评吧
一父教去赶人兵她然昏过前展道一：”退两清乱瞒弟兵时一般人里后芷家利了者是过意催看何甚此会我身点杖记到此是弹南说告隆真友这听侄他了两户　个：达，不，余，扯？些声，，”反个卫许话敢他她身子助来同里内竟的明但头此弦，不，何谁，的忘　深食着春半，龃完而总抗阁没向右等赵少情对放，家了是徐奔绮给也挥徐赵中忙和传声见咦天更来。是卫”，色见刚“出志公，那旁两好然花 
----
